## Fooling around to check what's going on

In [4]:
from pymongo import MongoClient
import pandas as pd
import json as js

In [5]:
def connectCollection(database, collection):
    client = MongoClient()
    db = client[database]
    coll = db[collection]
    return db, coll

In [6]:
# looking for companies

## Haz las queries necesarias, como raised money, y coges esa columna y la pegas a este dataset de first_data, ( y que tengas en la query nombre y la fila que quieras, para que te lo indexe)

In [7]:
db, coll = connectCollection('companies','companies')

In [8]:
rest = list(coll.find())

In [13]:
first_data=pd.DataFrame(rest)
first_data["category_code"].value_counts()

web                 3787
software            2736
games_video         1083
mobile              1018
other                986
advertising          928
enterprise           742
ecommerce            688
consulting           637
network_hosting      626
public_relations     533
search               394
biotech              373
hardware             368
cleantech            305
semiconductor        167
security             156
analytics             66
finance               49
social                49
news                  48
education             36
music                 33
messaging             30
legal                 25
medical               25
travel                25
photo_video           23
health                23
manufacturing         19
sports                13
fashion               10
real_estate           10
automotive             9
hospitality            8
transportation         7
nanotech               5
nonprofit              4
design                 4
local                  1


In [57]:
first_data.shape

(18801, 43)

In [9]:
print(len(rest))

NameError: name 'rest' is not defined

In [ ]:
# we found latitude and longitud
rest[0]['offices']

## GeoJSON

In [19]:
# mongo kind of sucks because this function depends on how was written the entry of a
# place. it could be in a different position than mentioned below, and we're doomed.
# i need to sharp this function a lot
def getLocation(place):
    
    longitude = place['offices'][0]["longitude"]
    latitude = place['offices'][0]["latitude"]
    loc = {
        'type':'Point',
        'coordinates':[longitude, latitude]
    }
    return loc

In [20]:
getLocation(rest[0])

{'type': 'Point', 'coordinates': [-122.333253, 47.603122]}

In [32]:

#si quiero añadir información a la colección $set (para el geoindex)

for place in rest:
    value = {"$set": {'location':getLocation(place)}}
    coll.update_one(place,value)
    # <collection>.update_one(<do

In [34]:
# Supongo que de la misma manera unset la elimina (mira bien esto)
for place in rest:
    value = {"$unset": {'address.coord':''}}
    coll.update_one(place,value)
    # <collection>.update_one(<document>,<value>)


In [25]:
rest = list(coll.find({'name':"Amazon"}))
list(rest[0])

['_id',
 'name',
 'permalink',
 'crunchbase_url',
 'homepage_url',
 'blog_url',
 'blog_feed_url',
 'twitter_username',
 'category_code',
 'number_of_employees',
 'founded_year',
 'founded_month',
 'founded_day',
 'deadpooled_year',
 'deadpooled_month',
 'deadpooled_day',
 'deadpooled_url',
 'tag_list',
 'alias_list',
 'email_address',
 'phone_number',
 'description',
 'created_at',
 'updated_at',
 'overview',
 'image',
 'products',
 'relationships',
 'competitions',
 'providerships',
 'total_money_raised',
 'funding_rounds',
 'investments',
 'acquisition',
 'acquisitions',
 'offices',
 'milestones',
 'ipo',
 'video_embeds',
 'screenshots',
 'external_links',
 'partners']

In [38]:
# para encontrar todo lo que hay en un radio (tienes que introducir las coord)
berlin_radio = coll.find(
    {"location":
     {"$near":
      {"$geometry":
       {"type":"Point",
        "coordinates":[122.3321, 47.6062]
       },
       "$maxDistance":10000
      }
     }
    }
)
berlin_radio=list(berlin_radio)

OperationFailure: error processing query: ns=companies.companiesTree: GEONEAR  field=location maxdist=10000 isNearSphere=0
Sort: {}
Proj: {}
 planner returned error :: caused by :: unable to find index for $geoNear query

In [31]:
import foli

OperationFailure: error processing query: ns=companies.companiesTree: GEONEAR  field=location maxdist=10000 isNearSphere=0
Sort: {}
Proj: {}
 planner returned error :: caused by :: unable to find index for $geoNear query